### TIFU KNN
Один из самых эффективных алгоритмов для решения задачи next basket recommendation  
Была использована его вариация  
Примерный алгоритм следующий:
* Каждую корзину кодируем как вектор, где на i-й позиции стоит количество i-го товара в этой корзине
* Далее складываем все корзины каждого пользователя:
    - Либо как взвешенное среднее по всем корзинам
    - Либо как сумму векторов, умноженных на веса
    - !Старые корзины имеют меньший коэффициент
* По полученным векторам находим ближайших соседей каждого пользователя и:
    - Либо берем итоговый вектор пользователя как взвешенное среднее по всем соседям и самому пользователю (чем дальше сосед, чем меньше коэффициент)
    - Либо находим взвешенное среднее по всем соседям, а дальше итоговый вектор получаем как P = v_user * alpha + v_avg_neighbors * (1 - alpha)
* Далее находим позиции в итоговом векторе каждого пользователя, где значения максимальны - это и есть рекомендации  
  
В оригинальном алгоритме корзины каждого пользователя разбиваются на группы, но так как в среднем покупок у пользователя не так много, этот шаг был убран

In [1]:
%%time
%pylab inline
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.neighbors import NearestNeighbors
from implicit.lmf import LogisticMatrixFactorization
from scipy import sparse
from tqdm import tqdm
import pickle
import gc
tqdm.pandas()

Populating the interactive namespace from numpy and matplotlib
Wall time: 2.49 s


In [2]:
df = pd.read_csv("train.csv")
ss = pd.read_csv("sample_submission.csv")

df.rename(columns={"order_completed_at":"time"}, inplace=True) # rename "order_completed_at" column to "time"
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # "time" column to datetime type

# Split "id" column to "user_id" and "cart" columns
ss["user_id"] = ss.id.progress_apply(lambda x: x.split(";")[0]).astype(int)
ss["cart"] = ss.id.progress_apply(lambda x: x.split(";")[1]).astype(int)
ss = ss[["user_id", "cart", "target"]] 

100%|██████████████████████████████████████████████████████████████████████| 790449/790449 [00:01<00:00, 661501.65it/s]


In [3]:
r = 0.8
k_nearest = 20              
alpha = 0.85    
top_k = 15

In [4]:
carts_count = 881
users_count = 20000

In [5]:
def duplicates_to_count(t):
    map_table = t.groupby("time")["user_id"].first().to_frame().reset_index()
    count_table = t.groupby("time")["cart"].value_counts().to_frame().rename(columns={"cart":"count"}).reset_index()
    return pd.merge(count_table, map_table, on="time", how="left").reset_index()[["user_id", "time", "cart", "count"]]

def count_to_duplicates(t):
    g = t.copy()
    g["to_explode"] = g["count"].apply(lambda x: [i for i in range(x)])
    g.explode("to_explode").reset_index().drop(columns=["count", "to_explode", "index"])
    return g

def to_required_df(df): # [user_id, cart, target] -> [id, target]
    df["id"] = df["user_id"].astype(str) + ";" + df["cart"].astype(str)
    df.drop(columns=["user_id", "cart"], inplace=True)
    df.rename(columns={"predict":"target"}, inplace=True)
    return df[["id", "target"]]

def make_train_targets(t, level=1):
    user_last_time = t.groupby(["user_id"])["time"].last().to_frame().reset_index()
    user_last_time["last_buy"] = 1
    
    train = pd.merge(t, user_last_time, on=["time", "user_id"], how="left")
    train = train[train["last_buy"] != 1]
    train.drop(columns=["last_buy"], inplace=True)
    
    if level >= 2:
        return make_train_targets(train, level-1)
    
    user_last_time.drop(columns=["last_buy"], inplace=True)
    
    user_last_carts = pd.merge(user_last_time, df.drop(columns=["user_id"]), on="time", how="left")
    
    skeleton = make_skeleton(train)
    targets = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")
    targets.fillna(0, inplace=True)
    targets["count"] = targets["count"].apply(lambda x: x if x <= 1 else 1).astype(int)
    targets.rename(columns={"count":"target"}, inplace=True)
    return train, targets

def make_skeleton(t):
    return t.groupby("user_id")["cart"].unique().to_frame().reset_index().explode("cart")

def user_cart_pivot_table(t):
    g = pd.pivot_table(t, columns="cart", index=["user_id", "time"], values="count", aggfunc=np.sum, fill_value=0)
    for cart in list(set(df["cart"].unique()).difference(set(t["cart"].unique()))):
        g[cart] = 0
    return g

In [6]:
df = duplicates_to_count(df)

In [7]:
train, targets = make_train_targets(df, level=1)

In [8]:
main = user_cart_pivot_table(train) # or user_cart_pivot_table(df) in test case

In [9]:
main = main.reindex(sorted(main.columns), axis=1).reset_index()

In [10]:
main.head()

cart,user_id,time,0,1,2,3,4,5,6,7,...,871,872,873,874,875,876,877,878,879,880
0,0,2020-07-19 09:59:17,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2020-08-24 08:55:32,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2019-05-08 16:09:41,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2020-01-17 14:44:23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2020-02-06 22:46:55,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
users_all_carts = [[] for i in range(users_count)]
main_array = main.to_numpy()
for arr in tqdm(main_array):
    users_all_carts[arr[0]].append(arr[2:])

100%|█████████████████████████████████████████████████████████████████████| 188155/188155 [00:00<00:00, 1103416.98it/s]


In [12]:
def get_users_one_cart_as_avg(users, r):
    users_as_avg_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        cur_weights = 0
        if len(users[i]) > 0:
            cart = pd.Series(users[i][0])
        for j in range(1, len(users[i])):
            weight = r**(len(users[i]) - j)
            cart += pd.Series(users[i][j]) * weight
            cur_weights += weight
        if cur_weights == 0:
            cur_weights = 1
        cart /= cur_weights
        users_as_avg_cart[i] = cart.tolist()
    return users_as_avg_cart

def get_users_one_cart_as_attenuation(users, r): #GOOD
    users_as_attenuation_cart = [[] for i in range(users_count)]
    for i in tqdm(range(len(users))):
        cart = pd.Series(np.zeros(carts_count), dtype=float)
        if len(users[i]) > 0:
            cart = pd.Series(users[i][0])
        for j in range(1, len(users[i])):
            cart += pd.Series(users[i][j]) * r**(len(users[i]) - j)
        users_as_attenuation_cart[i] = cart.tolist()
    return users_as_attenuation_cart

In [13]:
users_as_vectors = get_users_one_cart_as_attenuation(users_all_carts, r) ### YOU CAN CHANGE THE FUNCTION

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [01:30<00:00, 219.90it/s]


Ближайших соседей также можно искать несколькими способами, я выбрал два основных:
* NearestNeighbors из sklearn
* Матричное разложение из implicit

In [14]:
def get_neighbors_nn(users_carts, k): #GOOD
    model = NearestNeighbors()
    model.fit(users_carts)
    neighbors = model.kneighbors(users_carts, n_neighbors=k+1, return_distance=False)
    return neighbors

def get_neighbors_mf(users_carts, k):
    model = LogisticMatrixFactorization(iterations=100)
    model.fit(sparse.csr_matrix(users_carts).transpose())
    neighbors = [ [] for i in range(users_count)]
    for i in tqdm(range(users_count)):
        nbrs = model.similar_users(i, k+1)
        for tpl in nbrs:
            neighbors[i].append(tpl[0]) 
    return neighbors

In [15]:
neighbors = get_neighbors_mf(users_as_vectors, k_nearest) ### YOU CAN CHANGE THE FUNCTION

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:08<00:00, 2398.05it/s]


In [16]:
def combine_with_neighbors_avg(users_carts, neighbors, alpha):
    avg_nbrs = [ [] for i in range(users_count) ]
    for i in tqdm(range(users_count)):
        avg = pd.Series(np.zeros(carts_count))
        cur_weights = 0
        for j in range(1, len(neighbors[i])):
            avg += pd.Series(users_carts[neighbors[i][j]]) * j
            cur_weights += j
        avg /= (cur_weights if cur_weights != 0 else 1)
        avg_nbrs[i] = avg
    return [pd.Series(users_carts[i]) * alpha + pd.Series(avg_nbrs[i]) * (1-alpha) for i in range(users_count)]
    
def combine_with_neighbors_attenuation(users_carts, neighbors, alpha): #GOOD
    users_p = [ [] for i in range(users_count)]
    for i in tqdm(range(len(neighbors))):
        users_p[i] = pd.Series(users_carts[neighbors[i][0]]) * alpha
        for j in range(k_nearest):
            users_p[i] = users_p[i] + alpha**(j+2) * pd.Series(users_carts[neighbors[i][j+1]])
    return users_p

In [17]:
users_as_final_vectors = combine_with_neighbors_attenuation(users_as_vectors, neighbors, alpha) ### YOU CAN CHANGE THE FUNCTION

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [02:38<00:00, 126.23it/s]


In [18]:
users_best_basket = []
for i in tqdm(range(users_count)):
    users_best_basket.append(users_as_final_vectors[i].sort_values(ascending=False).head(top_k).index.tolist())

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:05<00:00, 3683.24it/s]


In [19]:
users_best_basket[:5]

[[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 84, 383, 441, 379, 409],
 [55, 14, 798, 89, 169, 17, 398, 804, 57, 22, 420, 41, 384, 383, 376],
 [57, 55, 14, 425, 82, 23, 84, 409, 61, 383, 412, 398, 54, 22, 430],
 [57, 398, 61, 409, 19, 84, 430, 22, 17, 23, 382, 399, 55, 14, 16],
 [61, 57, 14, 22, 55, 398, 17, 425, 409, 100, 54, 84, 420, 23, 99]]

Можно также добавлять к рекомендациям предметы, найденные статистически

In [20]:
add_carts_from_apriori = False

if add_carts_from_apriori:
    ap = pd.read_csv("apriori_top_20.csv")
    ap_map = {}
    for index, row in ap.iterrows():
        ap_map[int(row["from"])] = int(row["to"])
    
    for i in tqdm(range(users_count)):
        for key in ap_map.keys():
            if key in users_best_basket[i] and ap_map[key] not in users_best_basket[i]:
                users_best_basket[i].append(ap_map[key])

In [21]:
recs = pd.DataFrame()
recs["user_id"] = [i for i in range(users_count)]
recs["items"] = users_best_basket

In [22]:
res = pd.merge(make_skeleton(train), recs, on="user_id", how="left") # or ss[["user_id", "cart"]] instead of make_skeleton(train) in test

In [23]:
res.head()

,user_id,cart,items
0,0,14,"[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 8..."
1,0,20,"[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 8..."
2,0,57,"[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 8..."
3,0,82,"[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 8..."
4,0,379,"[14, 82, 41, 432, 57, 382, 23, 402, 398, 20, 8..."


In [24]:
res.isna().sum()

user_id    0
cart       0
items      0
dtype: int64

In [25]:
arr = []
res_array = res.to_numpy()
for row in tqdm(res_array):
    if type(row[2]) == float:
        arr.append(0)
        continue
    if row[1] in row[2]:
        arr.append(1)
    else:
        arr.append(0)
        
assert sum(arr) != 0
        
res["predict"] = arr

100%|████████████████████████████████████████████████████████████████████| 1032983/1032983 [00:06<00:00, 148326.43it/s]


In [26]:
res.drop(columns=["items"], inplace=True)

In [27]:
#In test case skip next 2 steps

In [28]:
res = pd.merge(res, targets, on=["user_id", "cart"], how="left")

In [29]:
f1_score(res["target"], res["predict"])

0.43724368708683037

#### Мини исследование алгоритмов
При фиксированных параметрах 
* r = 0.8
* k_nearest = 20              
* alpha = 0.85    
* top_k = 15    

Я менял варианты усреднения корзин пользователя, поиска ближайших соседей и объединения пользователя с соседями, получил следующие значения метрики
* avg-nn-avg - 0.43583
* avg-nn-att - 0.45276
* avg-mf-avg - 0.43530
* avg-mf-att - 0.43158
* att-nn-avg - 0.44374
* att-nn-att - 0.45488 - best
* att-mf-avg - 0.44441
* att-mf-att - 0.43928   

** att - attenuation - затухание, при таком подходе мы берем коэффициент затухания от новых покупок к старым  
Что можно сказать точно - для усреднения корзин пользователя лучше использовать get_users_one_cart_as_attenuation()

In [ ]:
# In test case

In [ ]:
res = to_required_df(res)

In [ ]:
t = pd.read_csv("user_top_10_rec_baseline.csv")

In [ ]:
L1 = t["target"].tolist()
L2 = res["target"].tolist()

In [ ]:
def diff(v1, v2):
    assert len(v1) == len(v2)
    k = 0
    for i in range(len(v1)):
        if v1[i] != v2[i]:
            k+=1
    return k/len(v1) * 100

In [ ]:
diff(L1, L2)

In [ ]:
res.to_csv("TIFU_KNN_k-{}_r-{}_aplha-{}_topk-{}_diff-{}_train-{}.csv".format(k_nearest, r, alpha, top_k, round(diff(L1, L2),2), 0.45488)
           , index=False)

In [ ]:
# r = 0.8
# k_nearest = 20  Local scores: 0.45276, 0.45954, 0.45952 - mean=0.45727             
# alpha = 0.85    Public score: 0.45760
# top_k = 15

In [ ]:
# r = 0.85
# k_nearest = 26    Local scores: 0.45541, 0.45957, 0.45913 - mean=0.45804 
# alpha = 0.91      Public score: 0.45462
# top_k = 14

In [ ]:
# r = 0.75
# k_nearest = 30    Local score: 0.4589, 0.46037, 0.46049 - mean=0.45992
# alpha = 0.95      Public score: 0.45338
# top_k = 18

In [ ]:
# r = 0.8
# k_nearest = 5    Local score: 0.43417 (0.43007)
# alpha = 0.75     Public score: 0.43433
# top_k = 10

In [ ]:
# r = 0.95
# k_nearest = 6    Local score: 0.42619
# alpha = 0.65     Public score: 0.42679
# top_k = 9

In [ ]:
# r = 0.8
# k_nearest = 20   Local score: 0.45276
# alpha = 0.85    
# top_k = 15

In [ ]:
# r = 0.73
# k_nearest = 15   Local score: 0.44712                       
# alpha = 0.79     Public score: 
# top_k = 15

In [ ]:
# r = 0.9
# k_nearest = 30  Local scores: 0.43557, 0.44458, 0.44311 - mean=0.44109
# alpha = 0.75    
# top_k = 10